In [83]:
import os
from os import listdir
from os.path import isfile, join
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import glob
import torch
import torch.cuda
import torch.nn.functional as F
from torch.utils.data import Dataset
import logging
import torchio as tio

from util.logconf import logging

log = logging.getLogger(__name__)
# log.setLevel(logging.WARN)
# log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

In [84]:
# pip install nibabel
# !pip install torchio

In [85]:
#id list
train_data_path = '/home/yingchihlin/Documents/Code/AISD/AISD.train'
id_list = os.listdir(train_data_path)
print(id_list[0])

os.path.join(train_data_path, id_list[0])
img_files = os.listdir(os.path.join(train_data_path, id_list[0]))
print(os.path.join(train_data_path, id_list[0]))
img_files

t_file = ('CT.nii.gz', 'mask.nii.gz')
print(os.path.join(train_data_path, id_list[0], t_file[0]))

0226311
/home/yingchihlin/Documents/Code/AISD/AISD.train/0226311
/home/yingchihlin/Documents/Code/AISD/AISD.train/0226311/CT.nii.gz


In [86]:
img_files

['mask.nii.gz', 'ICV.nii.gz', 'CT_ss.nii.gz', 'mask_clip.nii.gz', 'CT.nii.gz']

In [87]:
#0: CT, 1: Mask
filename = os.path.join(train_data_path, id_list[0], t_file[0])

img = nib.load(filename)
img.shape

data = img.get_fdata()
data.shape

(512, 512, 20)

In [88]:
import glob
uid = '0226311'
uid = id_list[0]
mhd_data_path = glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}/CT.nii.gz'.format(uid))[0]


img = nib.load(mhd_data_path)
data = img.get_fdata()
data.shape

mhd_mask_path = mhd_path = glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}/mask.nii.gz'.format(uid))[0]
img = nib.load(mhd_mask_path)
mask = img.get_fdata()
mask.shape

#clip
data.clip(-1000, 1000, data)
data

positive_indexes = (mask.sum(axis=(0,1)).nonzero()[0].tolist())
positive_indexes

[6, 11, 12, 13, 14, 15]

In [89]:
sample_list = []
sample_list += [(uid, slice_ndx) for slice_ndx in positive_indexes]
sample_list


[('0226311', 6),
 ('0226311', 11),
 ('0226311', 12),
 ('0226311', 13),
 ('0226311', 14),
 ('0226311', 15)]

In [90]:
data = np.moveaxis(data, -1, 0)
# data = data.permute(1, 2, 0)
data.shape

(20, 512, 512)

In [91]:
ndx = 1
contextSlices_count = 3
series_uid, slice_ndx = sample_list[ndx % len(sample_list)]
print(series_uid, slice_ndx)
ct_t = torch.zeros((contextSlices_count * 2 + 1, 512, 512))
print(ct_t.shape)
start_ndx = slice_ndx - contextSlices_count
end_ndx = slice_ndx + contextSlices_count + 1
print(start_ndx, end_ndx)

i = 0
context_ndx = 8
context_ndx = max(context_ndx, 0)
context_ndx = min(context_ndx, data.shape[2] - 1)
# print(context_ndx)

# print(data.shape)
# print(data[context_ndx].shape)
# ct_t[i] = torch.from_numpy(data[context_ndx].astype(np.float32))
# print(ct_t[i])


for i, context_ndx in enumerate(range(start_ndx, end_ndx)):
    context_ndx = max(context_ndx, 0)
    context_ndx = min(context_ndx, data.shape[2] - 1)
    ct_t[i] = torch.from_numpy(data[context_ndx].astype(np.float32))

# CTs are natively expressed in https://en.wikipedia.org/wiki/Hounsfield_scale
# HU are scaled oddly, with 0 g/cc (air, approximately) being -1000 and 1 g/cc (water) being 0.
# The lower bound gets rid of negative density stuff used to indicate out-of-FOV
# The upper bound nukes any weird hotspots and clamps bone down

ct_t.clamp_(-1000, 1000)

mask = np.moveaxis(mask, -1, 0)
pos_t = torch.from_numpy(mask[slice_ndx]).unsqueeze(0)
print(pos_t.shape)
pos_t

# pos_t = torch.from_numpy(ct.positive_mask[slice_ndx]).unsqueeze(0)

0226311 11
torch.Size([7, 512, 512])
8 15
torch.Size([1, 512, 512])


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)

In [92]:
#Dataset

def getCtSample(uid):
    #data
    mhd_data_path = mhd_path = glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}/CT.nii.gz'.format(uid))[0]
    img = nib.load(mhd_data_path)
    data = img.get_fdata()
    data = np.moveaxis(data, -1, 0)
    data.clip(-1000, 1000, data)

    #mask
    mhd_mask_path = mhd_path = glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}/mask.nii.gz'.format(uid))[0]
    img = nib.load(mhd_mask_path)
    mask = img.get_fdata()
    mask = np.moveaxis(mask, -1, 0)

    return data, mask


class CtDataset(Dataset):
    def __init__(self, uid, validation = False, val_stride = 5, contextSlices_count=3, fullCt_bool=True, id_list = id_list):

        self.contextSlices_count = contextSlices_count
        self.fullCt_bool = fullCt_bool
        self.series_list = id_list

        if  validation:
            assert val_stride > 0, val_stride
            self.series_list = self.series_list[::val_stride]
            assert self.series_list
        elif val_stride > 0:
            del self.series_list[::val_stride]
            assert self.series_list

        self.sample_list = []
        for series_uid in self.series_list:
            # print(series_uid)
            data, mask = getCtSample(series_uid)
            
            # positive slices
            positive_indexes = (mask.sum(axis=(0,1)).nonzero()[0].tolist())

            if self.fullCt_bool:
                self.sample_list += [(series_uid, slice_ndx)
                                     for slice_ndx in range(int(data.shape[0]))]
            else:
                self.sample_list += [(series_uid, slice_ndx)
                                     for slice_ndx in positive_indexes]


        logging.info("{!r}: {} {} series, {} slices".format(
            self,
            len(self.series_list),
            {None: 'general', False: 'training', True: 'validation'}[validation],
            len(self.sample_list)
        ))

        def __len__(self):
            return len(self.sample_list)

        def __getitem__(self, ndx):
            series_uid, slice_ndx = self.sample_list[ndx % len(self.sample_list)]
            return self.getitem_fullSlice(series_uid, slice_ndx)

        def getitem_fullSlice(self, series_uid, slice_ndx):
            ct, ct_mask = getCtSample(series_uid)

            #Full Slice
            if fullCt_bool:
                ct_t = ct

                # CTs are natively expressed in https://en.wikipedia.org/wiki/Hounsfield_scale
                # HU are scaled oddly, with 0 g/cc (air, approximately) being -1000 and 1 g/cc (water) being 0.
                # The lower bound gets rid of negative density stuff used to indicate out-of-FOV
                # The upper bound nukes any weird hotspots and clamps bone down
                ct_t.clamp_(-1000, 1000)

                pos_t = torch.from_numpy(ct_mask[slice_ndx]).unsqueeze(0)

            else:
                ct_t = torch.zeros((self.contextSlices_count * 2 + 1, 512, 512))

                start_ndx = slice_ndx - self.contextSlices_count
                end_ndx = slice_ndx + self.contextSlices_count + 1
                for i, context_ndx in enumerate(range(start_ndx, end_ndx)):
                    context_ndx = max(context_ndx, 0)
                    context_ndx = min(context_ndx, ct.shape[2] - 1)
                    ct_t[i] = torch.from_numpy(ct[context_ndx].astype(np.float32))

                ct_t.clamp_(-1000, 1000)

                pos_t = torch.from_numpy(ct_mask[slice_ndx]).unsqueeze(0)

            return ct_t, pos_t, series_uid, slice_ndx


In [93]:
dataset = CtDataset(id_list)

2022-09-27 13:51:48,942 INFO     pid:689774 root:051:__init__ <__main__.CtDataset object at 0x7f177d55f700>: 276 training series, 7465 slices


In [ ]:

class TrainingCtDataset(CtDataset):
    def __init__(self, uid, validation=False, val_stride=5, contextSlices_count=3, fullCt_bool=False, id_list=...):
        super().__init__(uid, validation, val_stride, contextSlices_count, fullCt_bool, id_list)

        self.ratio_int = 2

    def __len__(self):
        return 346*(1-self.val_stride)/self.val_stride

    def __getitem__(self, ndx):
        candidateInfo_tup = self.pos_list[ndx % len(self.pos_list)]
        return self.getitem_trainingCrop(candidateInfo_tup)

    def getitem_trainingCrop(self, candidateInfo_tup):
        ct_a, pos_a, center_irc = getCtRawCandidate(
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            (7, 96, 96),
        )
        pos_a = pos_a[3:4]  #middle mask slice as the training label

        row_offset = random.randrange(0,32)
        col_offset = random.randrange(0,32)
        ct_t = torch.from_numpy(ct_a[:, row_offset:row_offset+64,
                                     col_offset:col_offset+64]).to(torch.float32)
        pos_t = torch.from_numpy(pos_a[:, row_offset:row_offset+64,
                                       col_offset:col_offset+64]).to(torch.long)

        slice_ndx = center_irc.index

        return ct_t, pos_t, candidateInfo_tup.series_uid, slice_ndx

    
